In [2]:
from awpy import Demo

import torch
from torch_geometric_temporal.signal import DynamicHeteroGraphTemporalSignal

import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from termcolor import colored
import time
import json
import sys
import os

pd.set_option('display.max_columns', 100)
pd.set_option('future.no_silent_downcasting', True)

pl.Config.set_tbl_rows(10)

sys.path.append(os.path.abspath('../../package'))

from CS2.graph import TabularGraphSnapshot, HeteroGraphSnapshot, TemporalHeteroGraphSnapshot
from CS2.token import Tokenizer
from CS2.preprocess import Dictionary, NormalizePosition, NormalizeTabularGraphSnapshot, ImputeTabularGraphSnapshot
from CS2.visualize import HeteroGraphVisualizer

DATA_PATH = '../../data/matches-processed/cs2/hetero-graph'

# Create Temporal dataset

In [5]:
hetero_graphs = [f for f in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, f)) and f not in ['train.pt', 'val.pt']]

['100000.pt',
 '100001.pt',
 '100002.pt',
 '100003.pt',
 '100004.pt',
 '100005.pt',
 '100006.pt',
 '100007.pt',
 '100008.pt',
 '100009.pt',
 '100010.pt',
 '100011.pt',
 '100012.pt',
 '100013.pt',
 '100014.pt',
 '100015.pt',
 '100016.pt',
 '100017.pt',
 '100018.pt',
 '100019.pt',
 '100020.pt',
 '100021.pt',
 '100022.pt',
 '100023.pt',
 '100024.pt',
 '100025.pt',
 '100026.pt',
 '100027.pt',
 '100028.pt',
 '100029.pt',
 '100030.pt',
 '100031.pt',
 '100032.pt',
 '100033.pt',
 '100034.pt',
 '100035.pt',
 '100036.pt',
 '100037.pt',
 '100038.pt',
 '100039.pt',
 '100040.pt',
 '100041.pt',
 '100042.pt',
 '100043.pt',
 '100044.pt',
 '100045.pt',
 '100046.pt',
 '100047.pt',
 '100048.pt',
 '100049.pt',
 '100050.pt',
 '100051.pt',
 '100052.pt',
 '100053.pt',
 '100054.pt',
 '100055.pt',
 '100056.pt',
 '100057.pt',
 '100058.pt',
 '100059.pt',
 '100060.pt',
 '100061.pt',
 '100062.pt',
 '100063.pt',
 '100064.pt',
 '100065.pt',
 '100066.pt',
 '100067.pt',
 '100068.pt',
 '100069.pt',
 '100070.pt',
 '1000